In [48]:
!pip install requests
!pip install pandas
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
from pprint import pprint
key = '41b145a848f4bd67'
team_id = {'Spokane Chiefs': '215', 'Seattle Thunderbirds': '214', 'Portland Winterhawks': '208', 'Everett Silvertips': '226', 'Tri-City Americans': '217', 'Kamloops Blazers': '203', 'Kelowna Rockets': '204', 'Prince George Cougars': '210', 'Brandon Wheat Kings': '201', 'Swift Current Broncos': '216', 'Vancouver Giants': '223', 'Kootenay ICE': '222', 'Victoria Royals': '227', 'Medicine Hat Tigers': '206', 'Edmonton Oil Kings': '228', 'Moose Jaw Warriors': '207', 'Regina Pats': '212', 'Saskatoon Blades': '213', 'Prince Albert Raiders': '209', 'Calgary Hitmen': '202', 'Lethbridge Hurricanes': '205', 'Red Deer Rebels': '211', 'Winnipeg ICE': '222', 'Wenatchee Wild': '222'}

# URLs

In [3]:
def game_id_url_func(num_of_past_games, current_team_id):
  return f'https://lscluster.hockeytech.com/feed/?feed=modulekit&key={key}&view=scorebar&client_code=whl&numberofdaysahead=0&numberofdaysback={num_of_past_games}&season_id=&team_id={current_team_id}&lang_code=en&fmt=json'

def game_stats_url_func(game_id):
  return f'https://lscluster.hockeytech.com/feed/?feed=gc&key={key}&game_id={game_id}&client_code=whl&tab=clock&lang_code=en&fmt=json'

# Get Past Game IDs for each team

In [4]:
'''
Get the game IDs of the past x amount of games
'''
def get_game_ids(url):
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
  }

  try:
      response = requests.get(url, headers=headers)

      # Check if the request was successful
      if response.status_code == 200:

          return response.json()['SiteKit']['Scorebar']
          #pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")

## Get the Team ID of each opponent

### Get all teams past game IDs

In [41]:
# Keeps track of the game IDs that each team has played
team_games = {}
for team in team_id:    
  curr_game_id = game_id_url_func(num_of_past_games, team_id[team])
  game = get_game_ids(curr_game_id)
  team_games[team] = [x['ID'] for x in game]


In [44]:
print((team_games['Spokane Chiefs']))

['1015005', '1015015', '1015027', '1015029', '1015052', '1015063', '1015072', '1015080', '1015086', '1015098', '1015113', '1015132', '1015141', '1015145', '1015162', '1015166', '1015184', '1015205', '1015218', '1015227', '1015233', '1015244', '1015261', '1015269', '1015310', '1015286', '1015296', '1015302', '1015324', '1015334', '1015346', '1015352', '1015362', '1015371', '1015388', '1015396', '1015408', '1015411', '1015473', '1015474', '1015475', '1015476', '1015477', '1015478', '1015479', '1015533', '1015544', '1015552', '1015554', '1015572', '1015589', '1015609', '1015616', '1015629', '1015635', '1015649', '1015658', '1015669', '1015677', '1015690', '1015695', '1015700', '1015707', '1015733', '1015741', '1015752', '1015770', '1015773', '1015782', '1015790', '1015801', '1015812', '1015822', '1015828', '1015851', '1015864', '1015874', '1015882', '1015899', '1015906', '1015915', '1015920', '1015930', '1015938', '1015953', '1015968', '1015974', '1015986', '1016002', '1016009', '1016026'

# Get the stats from specific games

## Initilize the DataFrame

In [29]:
import pandas as pd
df_columns = ["Game_ID", "Home_Name", "Away_Name", "Home_Goals", "Away_Goals", "Home_PP%", "Away_PP%", "Home_SOG", "Away_SOG", "Home_FOW%", "Away_FOW%"]
dataset = pd.DataFrame(columns=df_columns)

## Function to fetch team stats, given a game ID

In [37]:
'''
Expects a game_id, which is used to access the Hockey Tech API to get the stats of a game, returned in a JSON format
'''
def get_game_stats(game_id):
  game_stats_url = game_stats_url_func(game_id)
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
  }
  
  try:
      response = requests.get(game_stats_url, headers=headers)
  
      # Check if the request was successful
      if response.status_code == 200:
  
          return response.json()
          pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")

## Parse through each game stats from each team

In [57]:
from tqdm import tqdm

# Calculate total number of games for the overall progress bar
total_games = sum(len(team_games[team]) for team in team_games)

# Keep track of games we've already seen
existing_game_ids = set()

with tqdm(total=total_games, desc="Processing all games", unit="game") as overall_pbar:
    # Go through each team
  for team in team_games:
    for game in team_games[team]:
      # Only get game stats if we haven't fetched it already
      if game in existing_game_ids:
          overall_pbar.update(1)  # Update the overall progress bar even if skipped
          continue

      existing_game_ids.add(game)
        
      # Get the stats of the game ID
      stats = get_game_stats(game)
  
      # Home and visitor team names
      visitor = stats['GC']['Clock']['visiting_team']['name']
      home = stats['GC']['Clock']['home_team']['name']
  
      # Number of goals
      home_goals = int(stats['GC']['Clock']['home_goal_count'])
      visitor_goals = int(stats['GC']['Clock']['visiting_goal_count'])
  
      # Calculate Power Play %
      home_ppp_total = float(stats['GC']['Clock']['power_play']['total']['home'])
      visitor_ppp_total = float(stats['GC']['Clock']['power_play']['total']['visiting'])
      # Avoid division by zero for home_ppp
      if home_ppp_total != 0:
          home_ppp = float(stats['GC']['Clock']['power_play']['goals']['home']) / home_ppp_total
      else:
          home_ppp = 0  # Default to 0 if no power plays
      # Avoid division by zero for visitor_ppp
      if visitor_ppp_total != 0:
          visitor_ppp = float(stats['GC']['Clock']['power_play']['goals']['visiting']) / visitor_ppp_total
      else:
          visitor_ppp = 0  # Default to 0 if no power plays
  
      # Calculate Faceoff Win %
      home_fowp = float(stats['GC']['Clock']['fow']['home'])
      visitor_fowp = float(stats['GC']['Clock']['fow']['visiting'])
      fow_total = home_fowp + visitor_fowp
      if fow_total != 0:
          home_fowp /= fow_total
          visitor_fowp /= fow_total
      else:
          home_fowp, visitor_fowp = .5, .5
  
      # Shots on goal
      home_sog = sum(stats['GC']['Clock']['shots_on_goal']['home'].values())
      visitor_sog = sum(stats['GC']['Clock']['shots_on_goal']['visiting'].values())
      
      # Add data to the dataset
      dataset.loc[len(dataset)] = {
          "Game_ID": game,
          "Home_Name": home,
          "Away_Name": visitor,
          "Home_Goals": home_goals,
          "Away_Goals": visitor_goals,
          "Home_PP%": home_ppp,
          "Away_PP%": visitor_ppp,
          "Home_SOG": home_sog,
          "Away_SOG": visitor_sog,
          "Home_FOW%": home_fowp,
          "Away_FOW%": visitor_fowp
      }
  
      # Update progress bar
      overall_pbar.update(1)


Processing all games:   0%|▏                                                    | 40/12004 [00:19<1:38:10,  2.03game/s]


KeyboardInterrupt: 

## Display and Write CSV File

In [52]:
# Show the dataset
display(dataset)

# Count how many times 'Spokane Chiefs' is in dataset
print((dataset['Home_Name'] == "Spokane Chiefs").suwm() + (dataset['Away_Name'] == "Spokane Chiefs").sum())

# Write to comma separated values file
dataset.to_csv('All_teams_WHL_stats.csv')

,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
0,1015005,Spokane Chiefs,Seattle Thunderbirds,5,4,0.166667,0.666667,42,29,0.550725,0.449275
1,1015015,Seattle Thunderbirds,Spokane Chiefs,5,4,0.000000,0.000000,26,33,0.584906,0.415094
2,1015027,Spokane Chiefs,Tri-City Americans,4,3,0.250000,0.000000,45,24,0.508475,0.491525
3,1015029,Tri-City Americans,Spokane Chiefs,4,3,0.500000,0.250000,26,46,0.507463,0.492537
4,1015052,Spokane Chiefs,Portland Winterhawks,9,3,0.750000,0.000000,50,19,0.596154,0.403846
...,...,...,...,...,...,...,...,...,...,...,...
5825,1016244,Prince George Cougars,Spokane Chiefs,3,4,0.333333,0.000000,28,34,0.375000,0.625000
5826,1016249,Prince George Cougars,Spokane Chiefs,1,7,0.125000,0.333333,28,32,0.575758,0.424242
5827,1016257,Portland Winterhawks,Spokane Chiefs,2,5,0.166667,0.500000,52,35,0.474359,0.525641
5828,1016268,Tri-City Americans,Spokane Chiefs,4,5,0.500000,0.250000,34,36,0.557377,0.442623


AttributeError: 'Series' object has no attribute 'suwm'